# NLP16- Hugging Face :
# 목차

> LMS 따라해보기 

1. GLUE dataset과 Huggingface
2. 커스텀 프로젝트 제작 
* (1) Processor
* (2) Tokenizer와 Model
* (3) Train/Evaluation과 Test





> PROJECT : Custom Project + Hugging Face

* STEP 1. mnli 데이터셋을 분석
* STEP 2. MNLIProcessor클래스 구현
* STEP 3. 데이터셋 구성하기: processor 및 tokenizer(Huggingface) 활용
* STEP 4. model을 생성하여 학습 및 테스트

## 회고

# LMS 따라하기

##1.GLUE Benchmark Dataset

>Pretrained model의 성능을 측정: 
* 유명한 데이터셋 한 가지만 가지고 성능을 논하는 것이 아니라, classification, summarization, reasoning, Q&A 등 NLP 모델의 성능을 평가할 수 있는 다양한 task를 해당 모델 하나만을 이용해 모두 수행해 보면서 종합적인 성능을 논하는 것이 일반화

그중 NLP 모델의 성능을 측정하기 위한 데이터셋으로 최근 활용되는 대표적인 것 중

 General Language Understanding Evaluation(GLUE) benchmark Dataset이 있습니다. (총 10가지 데이터셋)

* https://gluebenchmark.com/

* CoLA : 문법에 맞는 문장인지 판단
* MNLI : 두 문장의 관계 판단(entailment, contradiction, neutral)
* MNLI-MM : 두 문장이 안 맞는지 판단
* MRPC : 두 문장의 유사도 평가
* SST-2 : 감정분석
* STS-B : 두 문장의 유사도 평가
* QQP : 두 질문의 유사도 평가
* QNLI : 질문과 paragraph 내 한 문장이 함의 관계(entailment)인지 판단
* RTE : 두 문장의 관계 판단(entailment, not_entailment)
* WNLI : 원문장과 대명사로 치환한 문장 사이의 함의 관계 판단


In [ ]:
! pip install transformers

In [ ]:
! pip install datasets
# ! pip install huggingface-hub==0.1.0

In [ ]:
# ! python examples/tensorflow/text-classification/run_glue.py \
# 	--model_name_or_path bert-base-cased \
# 	--task_name mrpc \
# 	--output_dir ./models/mrpc \
# 	--overwrite_output_dir \
# 	--do_train \
# 	--do_eval \
# 	--num_train_epochs 1 \
# 	--save_steps 20000

 코드는 10가지 GLUE task 중 'mrpc' task를 수행하는 예제 코드입니다. 이 코드는 Huggingface의 framework 기반으로 BERT bert-base-cased을 활용하여 'mrpc' task를 수행합니다. 만약 task_name 및 다른 파라미터를 적절히 변경한 후 수행하면 다른 GLUE task도 간단히 수행해 볼 수 있을 것입니다. model도 다양하게 바꾸어 보면서 수월하게 수행 가능할 것입니다.

## 2.커스텀 프로젝트 제작 

In [ ]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures


### (1) Processor


#### mrpc 데이터셋 분석

In [ ]:
# tensorflow_datasets에서 제공하는 데이터
data, info = tfds.load('glue/mrpc', with_info=True)
info.splits['train'].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3668 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/glue/mrpc/2.0.0.incompleteRQNJRE/glue-train.tfrecord*...:   0%|          | 0/3…

...

Shuffling ~/tensorflow_datasets/glue/mrpc/2.0.0.incompleteRQNJRE/glue-validation.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/1725 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/glue/mrpc/2.0.0.incompleteRQNJRE/glue-test.tfrecord*...:   0%|          | 0/17…

Dataset glue downloaded and prepared to ~/tensorflow_datasets/glue/mrpc/2.0.0. Subsequent calls will reuse this data.


3668

In [ ]:
# tf.data.Dataset을 상속받은 클래스의 형태
# 1개의 데이터만 가져다가 어떻게 생겼는지 확인
data['train'].take(1)

<TakeDataset element_spec={'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sentence1': TensorSpec(shape=(), dtype=tf.string, name=None), 'sentence2': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
#  데이터셋 안의 정의된 항목 확인
examples = data['train'].take(1)
for example in examples:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['label']
    print(sentence1)
    print(sentence2)
    print(label)

tf.Tensor(b'The identical rovers will act as robotic geologists , searching for evidence of past water .', shape=(), dtype=string)
tf.Tensor(b'The rovers act as robotic geologists , moving on six wheels .', shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


#### 방법1. Processor의 활용

추상클래스인 Processor를 한번 상속받은 후, Sequence Classification task를 수행하는 모델의 Processor 추상클래스인 DataProcessor

In [ ]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))



 'mrpc' 원본 데이터셋을 처리하여 모델에 입력할 수 있도록 정리해 주는 MrpcProcessor 클래스

In [ ]:
class MrpcProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence1"].numpy().decode("utf-8"),
            tensor_dict["sentence2"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


> get_example_from_tensor_dict(): 
* Processor: 'Raw Dataset를 Annotated Dataset으로 **'변환'** 하는 역할

In [ ]:
processor = MrpcProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

------원본데이터------
{'idx': <tf.Tensor: shape=(), dtype=int32, numpy=1680>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'sentence1': <tf.Tensor: shape=(), dtype=string, numpy=b'The identical rovers will act as robotic geologists , searching for evidence of past water .'>, 'sentence2': <tf.Tensor: shape=(), dtype=string, numpy=b'The rovers act as robotic geologists , moving on six wheels .'>}
------processor 가공데이터------
InputExample(guid=1680, text_a='The identical rovers will act as robotic geologists , searching for evidence of past water .', text_b='The rovers act as robotic geologists , moving on six wheels .', label='0')


>`tfds_map()`: 
* label을 가공하는 메소드인데, 이미 숫자로 잘 가공된 label이기 때문에 특별한 변화가 없다.

In [ ]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=1680, text_a='The identical rovers will act as robotic geologists , searching for evidence of past water .', text_b='The rovers act as robotic geologists , moving on six wheels .', label='0')


Binary Classification 문제로 잘 정의되고 있는지 확인

In [ ]:
label_list = processor.get_labels()
label_list

['0', '1']

In [ ]:
label_map = {label: i for i, label in enumerate(label_list)}
label_map

{'0': 0, '1': 1}

#### 방법2. huggingface datasets 활용

Huggingface에서 제공하는 datasets를 이용 : 쉽고 간편한 방법
*  DataProcessor를 사용할 필요없이 바로 사용가능하게 구성

In [ ]:
import datasets
from datasets import load_dataset

huggingface_mrpc_dataset = load_dataset('glue', 'mrpc')
print(huggingface_mrpc_dataset)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


###(2) Tokenizer와 Model


#### 방법1. 클래스MRPCProcessor + framework 결합

In [ ]:
# tokenizer와 model 간단히 생성
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

`_glue_convert_examples_to_features()` 
*  processor가 생성한 example을 tokenizer로 인코딩하여 feature로 변환하는 역할 

In [ ]:
#  model에 입력할 데이터셋을 생성: processor와 tokenizer, 원본 데이터셋 결합

def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features


`tf_glue_convert_examples_to_features() `
* 내부적으로 `_glue_convert_examples_to_features()`를 호출해서 얻은 feature를 바탕으로 tf.data.Dataset을 생성하여 리턴

In [ ]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )


 tf.data.Dataset 인스턴스를 생성 : 생성된 학습 단계 데이터셋 train_dataset

In [ ]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)

Using label list ['0', '1']
*** Example ***
guid: 1680
features: InputFeatures(input_ids=[101, 1996, 7235, 9819, 2097, 2552, 2004, 20478, 21334, 2015, 1010, 6575, 2005, 3350, 1997, 2627, 2300, 1012, 102, 1996, 9819, 2552, 2004, 20478, 21334, 2015, 1010, 3048, 2006, 2416, 7787, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, lab

In [ ]:
examples = train_dataset.take(1)
for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101,  1996,  7235,  9819,  2097,  2552,  2004, 20478, 21334,
        2015,  1010,  6575,  2005,  3350,  1997,  2627,  2300,  1012,
         102,  1996,  9819,  2552,  2004, 20478, 21334,  2015,  1010,
        3048,  2006,  2416,  7787,  1012,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  

* 데이터셋 구성

In [ ]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

Using label list ['0', '1']
*** Example ***
guid: 1680
features: InputFeatures(input_ids=[101, 1996, 7235, 9819, 2097, 2552, 2004, 20478, 21334, 2015, 1010, 6575, 2005, 3350, 1997, 2627, 2300, 1012, 102, 1996, 9819, 2552, 2004, 20478, 21334, 2015, 1010, 3048, 2006, 2416, 7787, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, lab

In [ ]:
# validation 데이터셋
validation_dataset = tf_glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

Using label list ['0', '1']
*** Example ***
guid: 3155
features: InputFeatures(input_ids=[101, 1996, 2265, 1005, 1055, 8503, 5360, 2353, 1011, 4284, 16565, 2566, 3745, 2011, 1037, 10647, 1012, 102, 1996, 2194, 2056, 2023, 19209, 16565, 2011, 1037, 10647, 1037, 3745, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, label=1)

In [ ]:
# test 데이터셋
test_dataset = tf_glue_convert_examples_to_features(data['test'], tokenizer, max_length=128, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(16)

Using label list ['0', '1']
*** Example ***
guid: 163
features: InputFeatures(input_ids=[101, 6661, 1999, 8670, 2020, 2091, 1015, 1012, 1019, 3867, 2012, 16923, 7279, 3401, 2011, 16087, 2692, 13938, 2102, 1010, 2125, 1037, 2659, 1997, 17943, 2361, 1010, 1999, 1037, 3621, 6428, 3452, 2414, 3006, 1012, 102, 6661, 1999, 8670, 2020, 2091, 2093, 3867, 2012, 13913, 1011, 1015, 1013, 1018, 7279, 3401, 2011, 5641, 22394, 13938, 2102, 1010, 2125, 1037, 2659, 1997, 17943, 7279, 3401, 1010, 1999, 1037, 6428, 3006, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

#### 방법2. Huggingface Auto Classes 활용

Huggingface : AutoTokenizer, AutoModel 기능 제공(Auto Class 지원)

`Auto class` : `from_pretrained() `메소드를 이용해 **""pretrained model의 경로 혹은 이름"**만 안다면 자동으로 생성하는 방법

다양한 모델에 자동으로 맞출 수 있기 떄문에 특정 task와 dataset이 주어져있는 경우 모델을 다양하게 넣어 실험 가능

* `AutoTokenizer`: 예를들어 BERT모델은 BERT로 RoBERTa모델은 RoBERTa로 자동 변환

* `AutoModel`: 그대로 사용하기보다 특정 task를 지정하는 방식인 AutoModelForSequenceClassification을 사용하는걸 권장


In [ ]:
# Tokenizer , model 생성 
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2)

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

* 토크나이징: transform이라는 함수를 만들고 이전에 만들어두었던 Tokenizer를 사용하는데 이때 dataset의 형태를 확인하고 바꿀 대상을 지정

* mrpc:  sentence1, sentence2가 토크나이징할 대상이므로 data[’sentence1’], data[’sentence2’]로 인덱싱해서 지정

* truncation: 특정 문장이 길어 모델을 다루기 힘들어 질 수 있으므로 짧게 자르는 것을 의미

* return_token_type_ids: 문장이 한개이상일 때 나뉨 (해당 내용은 task에 필요없으므로 제거)



In [ ]:
def transform(data):
  return huggingface_tokenizer(
      data['sentence1'],
      data['sentence2'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = False,
      )
  
examples = huggingface_mrpc_dataset['train'][:5]
examples_transformed = transform(examples)

print(examples)
print(examples_transformed)

{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .', 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .', 'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .'], 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .", "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .", 'Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .', 'P

map: 데이터셋을 한번에 토크나이징할때 자주 사용하는 기법
* Data dictionary에 있는 모든 데이터들이 빠르게 적용 가능
* map을 사용해 토크나이징을 진행하기 때문에 batch를 적용해야 되므로 batched=True 설정

In [ ]:
encoded_dataset = huggingface_mrpc_dataset.map(transform, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

### (3) Train/Evaluation과 Test

#### 방법1. tf.keras.model 활용한 학습

` model.fit()`: 모델학습을 진행

In [ ]:
num_classes = len(processor.get_labels())

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
model.fit(train_dataset_batch, epochs=3, steps_per_epoch=115, 
                validation_data=validation_dataset_batch)

Epoch 1/3
115/115 [==============================] - 39s 236ms/step - loss: 0.5891 - acc: 0.6826 - val_loss: 0.4973 - val_acc: 0.7255
Epoch 2/3
115/115 [==============================] - 25s 219ms/step - loss: 0.4532 - acc: 0.7845 - val_loss: 0.3786 - val_acc: 0.8309
Epoch 3/3
115/115 [==============================] - 26s 222ms/step - loss: 0.3678 - acc: 0.8304 - val_loss: 0.3872 - val_acc: 0.8431


In [ ]:
result = model.evaluate(test_dataset_batch)
print(result)

108/108 [==============================] - 9s 78ms/step - loss: 0.4883 - acc: 0.7032
[0.48831719160079956, 0.703188419342041]


In [ ]:
import os
output_dir = '/content/drive/MyDrive/Aiffel_data/NLP/[NLP-16]/transformers'
output_eval_file = os.path.join(output_dir, "eval_results.txt")

with open(output_eval_file, "w") as writer:
    for i, v in enumerate(result) :
        if i == 0 :
            writer.write("Loss = %f\t" %(v))
        if i == 1 :
            writer.write("Accuracy = %f\n" %(v))
print("완료=3")

#파일에 쓴 테스트 결과 확인
# !cat ~/aiffel/transformers/eval_results.txt

완료=3


#### 방법2. Trainer 활용한 학습

In [ ]:
#메모리를 비워줍니다.
del model
del train_dataset_batch
del validation_dataset_batch
del test_dataset_batch

In [ ]:
# Trainer을 활용하는 형태로 모델 재생성
from transformers import Trainer, TrainingArguments
output_dir = '/content/drive/MyDrive/Aiffel_data/NLP/[NLP-16]/transformers'
metric_name = 'accuracy'

training_arguments = TrainingArguments(
    output_dir, # output이 저장될 경로
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, #learning_rate
    per_device_train_batch_size = 16, # 각 device 당 batch size
    per_device_eval_batch_size = 16, # evaluation 시에 batch size
    num_train_epochs = 3, # train 시킬 총 epochs
    weight_decay = 0.01, # weight decay
)

In [ ]:
from datasets import load_metric
metric = load_metric('glue', 'mrpc')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


In [ ]:
trainer = Trainer(
    model=huggingface_model,                           # 학습시킬 model
    args=training_arguments,                  # TrainingArguments을 통해 설정한 arguments
    train_dataset=encoded_dataset['train'],    # training dataset
    eval_dataset=encoded_dataset['validation'],       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2. If idx, sentence1, sentence2 are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 690
  Number of trainable parameters = 66955010


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.437380,0.808824,0.864111
2,No log,0.374945,0.843137,0.891892
3,0.462400,0.391967,0.845588,0.893761


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2. If idx, sentence1, sentence2 are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2. If idx, sentence1, sentence2 are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Aiffel_data/NLP/[NLP-16]/transformers/checkpoint-500
Configuration saved in /content/drive/MyDrive/Aiffel_data/NLP/[NLP-16]/transformers/checkpoint-500/config.json
Model weights saved in

TrainOutput(global_step=690, training_loss=0.40861371911090355, metrics={'train_runtime': 531.1828, 'train_samples_per_second': 20.716, 'train_steps_per_second': 1.299, 'total_flos': 1457671254810624.0, 'train_loss': 0.40861371911090355, 'epoch': 3.0})

In [ ]:
trainer.evaluate(encoded_dataset['test'])

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2. If idx, sentence1, sentence2 are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1725
  Batch size = 16


{'eval_loss': 0.4304107129573822,
 'eval_accuracy': 0.8214492753623188,
 'eval_f1': 0.8706968933669186,
 'eval_runtime': 28.3067,
 'eval_samples_per_second': 60.94,
 'eval_steps_per_second': 3.815,
 'epoch': 3.0}

# PROJECT : Custom Project

 GLUE dataset의 mnli task를 수행하는 프로젝트를 커스텀 프로젝트 형태

 mnli task는 BERT를 사용하면 학습이 제대로 되지 않는다.
 * [Hugging Face](https://huggingface.co/models)

  링크에서 BERT가 아닌 다른 모델을 선택. tensorflow와 해당 모델에 대한 task로 검색하면 사용할 수 있는 모델이 나온다. 그 후 선택한 모델의 _tokenizer_와 해당 모델에 대한 task 와 모델 의 정보를 여기에서 찾아 각자의 프로젝트를 완성해 보자.

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow
import numpy
import transformers
import argparse

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(argparse.__version__)

2.9.2
1.21.6
4.24.0
1.1


In [ ]:
# import numpy as np
# from argparse import ArgumentParser
# import numpy as np
# import tensorflow as tf
# import tensorflow_datasets as tfds
# from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, AutoConfig
# from dataclasses import asdict
# from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures


## STEP 1. mnli 데이터셋 분석

>tensorflow-datasets를 이용하여 glue/mnli를 다운로드하려면 tensorflow-datasets 라이브러리 버전을 업그레이드 해야한다.

In [ ]:
! pip install tensorflow-datasets -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 36.9 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.6.0
    Uninstalling tensorflow-datasets-4.6.0:
      Successfully uninstalled tensorflow-datasets-4.6.0


In [ ]:
import os   #디렉토리 관리


import tensorflow as tf   #신경망
import tensorflow_datasets as tfds   #데이터셋
from dataclasses import asdict   #데이터 클래스


from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures   #허깅 페이스
from transformers import BertTokenizer, TFBertForSequenceClassification   #BERT
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification   #RoBerta

In [ ]:
dataset, info = tfds.load('glue/mnli', with_info=True)

print("=" * 100)
for idx, (domain, data) in enumerate(dataset.items()):
    print(f"{idx}: {domain}({len(data):,})")
print("=" * 100)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/5 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

Generating test_matched examples...:   0%|          | 0/9796 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/glue/mnli/2.0.0.incompleteHBRPOI/glue-test_matched.tfrecord*...:   0%|        …

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/glue/mnli/2.0.0.incompleteHBRPOI/glue-test_mismatched.tfrecord*...:   0%|     …

Dataset glue downloaded and prepared to ~/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.
0: train(392,702)
1: validation_matched(9,815)
2: validation_mismatched(9,832)
3: test_matched(9,796)
4: test_mismatched(9,847)


In [ ]:
print(dataset)

{Split('train'): <PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>, 'validation_matched': <PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>, 'validation_mismatched': <PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>, 'test_matched': <PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': Te

In [ ]:
dataset['train'].take(1)

<TakeDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
examples = dataset['train'].take(1)
for example in examples:
    Hypothesis = example['hypothesis']
    Premise = example['premise']
    Label = example['label']
    print(Hypothesis)
    print(Premise)
    print(Label)

tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)
tf.Tensor(b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)


In [ ]:
for data in dataset['train']:
    print("Hypothesis:", data['hypothesis'].numpy(), end="\n\n")
    print("Premise:", data['premise'].numpy(), end="\n\n")
    print("Label:", data['label'].numpy())
    break

Hypothesis: b'Meaningful partnerships with stakeholders is crucial.'

Premise: b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'

Label: 1


>검증 및 테스트 데이터 섞기 및 분할
* 검증(validation) 데이터와 테스트(test) 데이터를 섞어준 후, 다시 분할 하여 줍니다.
* 검증 데이터의 경우 matched 데이터와 mismatched 데이터를 결합하여 학습 시, 모델의 성능을 확인 합니다.

In [ ]:
def split_dataset(dataset, val_size):    
    dataset = dataset.shuffle(len(dataset))
    val_dataset = dataset.shuffle(len(dataset)).take(val_size)
    test_dataset = dataset.shuffle(len(dataset)).skip(val_size)
    return (val_dataset, test_dataset)


matched_data = dataset['test_matched'].concatenate(dataset['validation_matched'])
mismatched_data = dataset['test_mismatched'].concatenate(dataset['validation_mismatched'])

matched_val, matched_test = split_dataset(matched_data, len(dataset['validation_matched']))
mismatched_val, mismatched_test = split_dataset(mismatched_data, len(dataset['validation_mismatched']))

val = mismatched_val.concatenate(matched_val)


print("=" * 100)
print(f"matched_val Num: {len(matched_val):,}\tmismatched_val Num: {len(mismatched_val):,}")
print(f"matched_test Num: {len(matched_test):,}\tmismatched_test Num: {len(mismatched_test):,}")
print("=" * 100)

matched_val Num: 9,815	mismatched_val Num: 9,832
matched_test Num: 9,796	mismatched_test Num: 9,847


## STEP 2. MNLIProcessor클래스 구현

### Processor의 활용

In [ ]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

In [ ]:
class MnliProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "entailment"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


> `get_example_from_tensor_dict()`
* 'Raw Dataset를 Annotated Dataset으로 변환'하는 역할을 합니다. 
* 항목별로 text_a, text_b, label 등의 annotation이 포함된 InputExample로 변환

In [ ]:
processor = MnliProcessor()
examples = dataset['train'].take(1)

for example in examples:
    print('----------원본데이터-----------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('----------가공데이터-----------')
    print(example)

----------원본데이터-----------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
----------가공데이터-----------
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships wit

In [ ]:
label_list = processor.get_labels()
label_list

['contradiction', 'entailment', 'entailment']

## STEP 3. 데이터셋 구성
:processor 및 tokenizer(Huggingface) 활용


Transformers로부터 BERT와 RoBERTa의 토크나이저를 불러옵니다.

In [ ]:
# bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
# roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base", use_fast=True)

작은 용량의 모델과 토크나이저로 변경

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")
huggingface_model = AutoModelForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli")

Downloading:   0%|          | 0.00/258 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/776 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--typeform--distilbert-base-uncased-mnli/snapshots/996dacf8ea284d96ea21f88a345fd7d597de1f1f/config.json
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Model config DistilBertConfig {
  "_name_or_path": "typeform/distilbert-base-uncased-mnli",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "mnli",
  "hidden_dim": 3072,
  "id2label": {
    "0": "ENTAILMENT",
    "1": "NEUTRAL",
    "2": "CONTRADICTION"
  },
  "initializer_range": 0.02,
  "label2id": {
    "CONTRADICTION": 2,
    "ENTAILMENT": 0,
    "NEUTRAL": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--typeform--distilbert-base-uncased-mnli/snapshots/996dacf8ea284d96ea21f88a345fd7d597de1f1f/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--typeform--distilbert-base-uncased-mnli/snapshots/996dacf8ea284d96ea21f88a345fd7d597de1f1f/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--typeform--distilbert-base-uncased-mnli/snapshots/996dacf8ea284d96ea21f88a345fd7d597de1f1f/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--typeform--distilbert-base-uncased-mnli/snapshots/996dacf8ea284d96ea21f88a345fd7d597de1f1f/config.json
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.jso

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--typeform--distilbert-base-uncased-mnli/snapshots/996dacf8ea284d96ea21f88a345fd7d597de1f1f/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at typeform/distilbert-base-uncased-mnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


>`_glue_convert_examples_to_features() `
*  processor가 생성한 example을 tokenizer로 인코딩하여 feature로 변환하는 역할

In [ ]:
# 정수 인코딩
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features


>`tf_glue_convert_examples_to_features()` 
* 내부적으로 `_glue_convert_examples_to_features()`를 호출해서 얻은 feature를 바탕으로 tf.data.Dataset을 생성하여 리턴
* 최종적으로 모델에 전달될 tf.data.Dataset 인스턴스를 생성 (학습 단계 데이터셋 train_dataset)



In [ ]:
# tf_dataset

def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )


In [ ]:
def get_dataset_batch(raw_dataset_list, tokenizer, processor):
    dataset_list = []
    for idx, dataset in enumerate(raw_dataset_list):
        data = tf_glue_convert_examples_to_features(dataset, tokenizer, max_length=128, processor=processor)
        
        if idx == 0:
            data_batch = data.shuffle(100).batch(16).repeat(2)
        else:
            data_batch = data.shuffle(100).batch(16)
        dataset_list.append(data_batch)
    return dataset_list
    
    
raw_dataset_list = (dataset['train'], val, matched_test, mismatched_test)

dsbert_train, dsbert_val, dsbert_m_test, dsbert_mism_test = get_dataset_batch(
    raw_dataset_list, huggingface_tokenizer, processor
)

Using label list ['contradiction', 'entailment', 'entailment']
*** Example ***
guid: 16399
features: InputFeatures(input_ids=[101, 1999, 5038, 1997, 2122, 13136, 1010, 1048, 11020, 2038, 2499, 29454, 29206, 14626, 2144, 2786, 2000, 16636, 1996, 10908, 1997, 1996, 2110, 4041, 6349, 1998, 2000, 5323, 15902, 13797, 2007, 22859, 6461, 2012, 6469, 2075, 1037, 2047, 25353, 14905, 10735, 2483, 2090, 1996, 2976, 10802, 1998, 15991, 1997, 3423, 2578, 4804, 1012, 102, 15902, 13797, 2007, 22859, 2003, 10232, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# train_dataset = tf_glue_convert_examples_to_features(dataset['train'], huggingface_tokenizer, max_length=128, processor=processor)

In [ ]:
# # train 데이터셋
# train_dataset = tf_glue_convert_examples_to_features(dataset['train'], huggingface_tokenizer, max_length=128, processor=processor)
# train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

In [ ]:
# # validation 데이터셋
# validation_dataset = tf_glue_convert_examples_to_features(dataset['validation'], huggingface_tokenizer, max_length=128, processor=processor)
# validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

In [ ]:
# # test 데이터셋
# test_dataset = tf_glue_convert_examples_to_features(dataset['test'], huggingface_tokenizer, max_length=128, processor=processor)
# test_dataset_batch = test_dataset.shuffle(100).batch(16)

In [ ]:
examples = train_dataset.take(1)
for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101,  1996,  7235,  9819,  2097,  2552,  2004, 20478, 21334,
        2015,  1010,  6575,  2005,  3350,  1997,  2627,  2300,  1012,
         102,  1996,  9819,  2552,  2004, 20478, 21334,  2015,  1010,
        3048,  2006,  2416,  7787,  1012,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  

## STEP 4. model 생성, 학습 및 테스트

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

>## DISTILBERT 모델 생성
* 모델을 생성하고 앞서 정의한 옵티마이저를 적용
* 모델의 파라미터 크기는 109,484,547

In [ ]:
# huggingface_tokenizer = AutoTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")
# huggingface_model = AutoModelForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli")

In [ ]:
dsbert_model = TFBertForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli", num_labels=3)
dsbert_model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
dsbert_model.summary()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--typeform--distilbert-base-uncased-mnli/snapshots/996dacf8ea284d96ea21f88a345fd7d597de1f1f/config.json
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Model config BertConfig {
  "_name_or_path": "/tmp/mnli_distil_output/checkpoint-6000",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "mnli",
  "hidden_act": "gelu",
  "hidden_dim": 3072,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "ENTAILMENT",
    "1": "NEUTRAL",
    "2": "CO

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file tf_model.h5 from cache at /root/.cache/huggingface/hub/models--typeform--distilbert-base-uncased-mnli/snapshots/996dacf8ea284d96ea21f88a345fd7d597de1f1f/tf_model.h5
Some layers from the model checkpoint at typeform/distilbert-base-uncased-mnli were not used when initializing TFBertForSequenceClassification: ['pre_classifier', 'dropout_19', 'distilbert']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at typeform/distilbert-base-unca

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_57 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [ ]:
dsbert_train

<RepeatDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
type(dsbert_val)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [ ]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
dsbert_model.fit(dsbert_train, epochs=3, steps_per_epoch=115)
                #  validation_data = dsbert_val)

Epoch 1/3
115/115 [==============================] - 48s 412ms/step - loss: 0.6550 - acc: 0.6484
Epoch 2/3
115/115 [==============================] - 47s 410ms/step - loss: 0.6371 - acc: 0.6636
Epoch 3/3
115/115 [==============================] - 47s 413ms/step - loss: 0.6347 - acc: 0.6783


In [ ]:
matched_result = dsbert_model.evaluate(dsbert_m_test)
mismatched_result = dsbert_model.evaluate(dsbert_mism_test)

print(f"\nMatched Test data\tLoss: {matched_result[0]:.3f}\tAccuracy: {matched_result[1]:.3f}")
print(f"Mismatched Test data\tLoss: {mismatched_result[0]:.3f}\tAccuracy: {mismatched_result[1]:.3f}")


616/616 [==============================] - 93s 151ms/step - loss: 0.4929 - acc: 0.8284

Matched Test data	Loss: 0.501	Accuracy: 0.819
Mismatched Test data	Loss: 0.493	Accuracy: 0.828


>## BERT 모델 생성
* 모델을 생성하고 앞서 정의한 옵티마이저를 적용
* 모델의 파라미터 크기는 109,484,547 

In [ ]:
# bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
# bert_model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
# bert_model.summary()

> BERT 학습
* 학습 회수는 총 3회로 설정 합니다.

In [ ]:
# # 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
# bert_model.fit(train_dataset_batch, epochs=3, steps_per_epoch=115, 
#                 validation_data=val)

In [ ]:
# bert_model.fit(bert_train, epochs=3, steps_per_epoch=115, validation_data=bert_val)

> BERT 모델 성능 확인
* 학습 데이터에 포함된 도메인의 테스트 데이터(Matched)와 포함되지 않은 테스트 데이터(MisMatched)를 이용하여 모델의 성능을 확인 합니다.
* Matched 테스트 데이터의 손실값은 1.032이며, Mismatched 테스트 데이터의 손실값은 1.011 입니다.
* Matched 테스트 데이터의 정확도은 0.435이며, Mismatched 테스트 데이터의 정확도은 0.460 입니다.

In [ ]:
# result = model.evaluate(test_dataset_batch)
# print(result)

In [ ]:
# matched_result = bert_model.evaluate(bert_m_test)
# mismatched_result = bert_model.evaluate(bert_mism_test)

# print(f"\nMatched Test data\tLoss: {matched_result[0]:.3f}\tAccuracy: {matched_result[1]:.3f}")
# print(f"Mismatched Test data\tLoss: {mismatched_result[0]:.3f}\tAccuracy: {mismatched_result[1]:.3f}")

# 회고

### - 이번 프로젝트에서 **어려웠던 점**.
> 해결 못한 부분
* 모델이 너무 커서 BERT와 Roberta는 시도 하다가 학습에서 계속 멈춰 버렸다.
* 학습이 가능한 작은 모델을 찾아야 했다.

### - 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
> 알아낸 점
* 허깅페이스를 이용해서 model과 tokenizer를 불러서 모델을 학습시키고 평가 할 수 있다. 


> 아직 모호한점
*  tensorflow 를 이용해서 모델을 커스터마이징하는것이 사실 아직 어렵다. 



### - 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.


>#### **루브릭평가 지표**
>|번호|평가문항|상세기준|
>|:---:|---|---|
>|1| MNLI 데이터셋을 처리하는 `전용 Processor 클래스`를 정상적으로 구현하였다. |Processor 클래스에 대해 `1개 이상의 example에 대한 단위테스트`가 정상 진행되었다.|
>|2|  BERT tokenizer와 Processor를 결합하여 `데이터셋을 정상적으로 생성`하였다.| MNLI 데이터셋의 입력과 라벨의 정의에 잘 맞는 `tf.data.Dataset 인스턴스`가 얻어졌다.|
>|3|MNLI 데이터셋에 대해 `적당한 모델을 fine-tuning`하여 학습하였다.|모델 학습이 정상적으로 진행되었다.|


### - **자기 다짐**

* 코드 복기

### - **참고**
>참고 깃허브
* [자연어처리강의+ 허깅페이스](https://youtube.com/playlist?list=PLrLEKGJAgXxL-R9IqDH7HANWXRsS900tF)
* [참고노드](https://github.com/YAGI0423/aiffel_going_deeper_nlp/blob/main/going_deeper_16/GD16_v3_1.ipynb)


